In [1]:
%matplotlib ipympl
from mantid.simpleapi import *
from tkinter import *
import matplotlib.pyplot as plt
from mantid.plots.utility import MantidAxType
from matplotlib.colors import LightSource
from matplotlib import cm
import matplotlib as mpl
from mantid.api import AnalysisDataService as ADS
import functools
import scipy.io
import numpy as np
import pathlib
import math as m
from datetime import datetime as dt
from scipy.signal import find_peaks as fp
import os
import json
import shutil

This function takes the run, the experiment conducted, and the bank the user wishes to retrieve the data from.

In [1]:
def fileSearch(run,exp,bank,save=False,local=False,path=None):
    if local == True:
        dire,temp,sumSpec = localFileSearch(path=f'{path}',run=run,exp=exp,bank=bank,save=save)
        return dire,temp,sumSpec
    else:
        pass
    exps = ['Cryo','Furnace','Pressure','X Scan','Y Scan','Z Scan']
    currdir = os.getcwd() #This finds the folder that the script is being ran in
    savedir = os.path.join(currdir,'Saves') #This creates a new folder to save the data in
    newdir = os.path.join(savedir,'Runs')
    if not os.path.exists(newdir):
        os.makedirs(newdir)
    filedir = os.path.join(newdir,f'{run}')
    
    if len(str(run)) < 5:
        zero_diff = 5 - len(str(run))
        for i in range(zero_diff):
            run = f'0{run}' #This is for significantly early runs, such as 00001 and 00035 from 2002
            
    yr = dt.now().strftime('%y')
    cy = int(yr) + 1

    if bank == '1':
        sMin_One = 0
        sMax_One = 1200 #These are the spectrums to be provided to the load function. After a certain point,
        #the highest and lowest spectrum available to read are 1 and 2399 respectively, so this code factors
        #that in
        sMin_Two = 1
        sMax_Two = 1199
    elif bank == '2':
        sMin_One = 1201
        sMax_One = 2400
        sMin_Two = 1200
        sMax_Two = 2399
    elif bank == 'Both':
        sMin_One = 0
        sMax_One = 2400
        sMin_Two = 1
        sMax_Two = 2399
    else:
        sMin_One = 0
        sMax_One = 2400
        sMin_Two = 1
        sMax_Two = 2399
    for i in reversed(range(2,int(cy))):
        if i < 10:
            year = f'0{i}'
        else:
            year = f'{i}'
        if i < 17:
            #These are the names of all the parameters saved as part of the raw file or folder prior
            #to the apparent overhaul in 2017
            apps = [['cryo_1','cryo_temp1'],'furnace','position','x','y','z']
            if exp == exps[0]:
                if i < 10:
                    app = apps[0][0]
                else:
                    app = apps[0][1]
            elif exp == exps[1]:
                app = apps[1]
            elif exp == exps[2]:
                app = apps[2]
            elif exp == exps[3]:
                app = apps[3]
            elif exp == exps[4]:
                app = apps[4]
            elif exp == exps[5]:
                app = apps[5]
            elif exp == None:
                pass
            else:
                raise NameError(f'This is not included in the list of experiments. Please enter one from this list: {exps}')
        else:
            #These are the current names for the parameters found within the nexus files
            apps = ['Temp_1','Temp_3','position','X_position','Y_position','Z_position']
            if exp == exps[0]:
                app = apps[0]
            elif exp == exps[1]:
                app = apps[1]
            elif exp == exps[2]:
                app = apps[2]
            elif exp == exps[3]:
                app = apps[3]
            elif exp == exps[4]:
                app = apps[4]
            elif exp == exps[5]:
                app = apps[5]
            elif exp == None:
                pass
            else:
                raise NameError(f'This is not included in the list of experiments. Please enter one from this list: {exps}')
        for cycle in [5,4,3,2,1]:
            if i < 8:
                name_one = f'ENG{run}'
                name_two = f'ENG{run}' #This factors in a change in naming convention
            else:
                name_one = f'ENG00{run}'
                name_two = f'ENGINX00{run}'
            ext_one = '.raw'
            ext_two = '.nxs'
            #These four combinations factor in the naming convention of runs mentioned above and the filetype,
            #either nexus or raw
            testdir_one = f'//isis/inst$/NDXENGINX/Instrument/data/cycle_{year}_{cycle}/{name_one}{ext_one}'
            testdir_two = f'//isis/inst$/NDXENGINX/Instrument/data/cycle_{year}_{cycle}/{name_two}{ext_one}'
            testdir_three = f'//isis/inst$/NDXENGINX/Instrument/data/cycle_{year}_{cycle}/{name_one}{ext_two}'
            testdir_four = f'//isis/inst$/NDXENGINX/Instrument/data/cycle_{year}_{cycle}/{name_two}{ext_two}'
            check_one = pathlib.Path(testdir_one)
            check_two = pathlib.Path(testdir_two)
            check_three = pathlib.Path(testdir_three)
            check_four = pathlib.Path(testdir_four)
            if check_one.exists():
                dire = testdir_one
                name = f'ENGINX00{run}'
                dSpace = f'{name}-dSpacing'
                sumSpec = f'{name}-sumSpec'
                try:
                    Load(Filename=dire, OutputWorkspace=name, SpectrumMin=sMin_One, SpectrumMax=sMax_One)
                except ValueError:
                    Load(Filename=dire, OutputWorkspace=name, SpectrumMin=sMin_Two, SpectrumMax=sMax_Two) #For the
                    #different spectrums, mentioned earlier
                ConvertUnits(InputWorkspace=name, OutputWorkspace=dSpace, Target='dSpacing', AlignBins=True)
                SumSpectra(InputWorkspace=dSpace, OutputWorkspace=sumSpec)
                try:
                    #It searches for the parameter in the file
                    temp = mtd[sumSpec].getRun().getLogData(app).filtered_value
                except RuntimeError:
                    print(f'Unable to find {app} in file.')
                    try:
                        #If this doesn't work, it searches for the parameter in the folder
                        temp_fp = f'//isis/inst$/NDXENGINX/Instrument/data/cycle_{year}_{cycle}/{name_one}_{app}.txt'
                        temp = np.loadtxt(fname=temp_fp,dtype='float',usecols=(1))
                        if save == True:
                            try:
                                shutil.copy(temp_fp,f'{newdir}')
                            except PermissionError:
                                print('Unable to copy parameter file. May already be saved.')
                    except FileNotFoundError:
                        #If this still doesn't work, it notifies the user and carries on
                        print(f'Unable to find {app} file in folder.')
                        temp = 0
                except UnboundLocalError:
                    temp = 0
                if isinstance(temp, (np.ndarray, list)) == True:
                    try:
                        #This is due to the nature of the txt files some parameters are read from holding more
                        #information than just the parameter the user would like to retrieve
                        temp = temp[-1]
                        temp = round(temp,2)
                    except IndexError:
                        temp = temp
                        temp = round(float(temp),2)
                else:
                    temp = temp
                    temp = round(float(temp),2)
                if save == True:
                    try:
                        shutil.copy(dire,f'{newdir}')
                    except PermissionError:
                        print('Unable to copy file. May already be saved.')
                return dire, temp, sumSpec
            elif check_two.exists():
                #It then repeats this process for every potential name till it finds a fit
                dire = testdir_two
                name = f'ENGINX00{run}'
                dSpace = f'{name}-dSpacing'
                sumSpec = f'{name}-sumSpec'
                try:
                    Load(Filename=dire, OutputWorkspace=name, SpectrumMin=sMin_One, SpectrumMax=sMax_One)
                except ValueError:
                    Load(Filename=dire, OutputWorkspace=name, SpectrumMin=sMin_Two, SpectrumMax=sMax_Two)
                ConvertUnits(InputWorkspace=name, OutputWorkspace=dSpace, Target='dSpacing', AlignBins=True)
                SumSpectra(InputWorkspace=dSpace, OutputWorkspace=sumSpec)
                try:
                    temp = mtd[sumSpec].getRun().getLogData(app).filtered_value
                except RuntimeError:
                    print(f'Unable to find {app} in file.')
                    try:
                        temp_fp = f'//isis/inst$/NDXENGINX/Instrument/data/cycle_{year}_{cycle}/{name_two}_{app}.txt'
                        temp = np.loadtxt(fname=temp_fp,dtype='float',usecols=(1))
                        if save == True:
                            try:
                                shutil.copy(temp_fp,f'{newdir}')
                            except PermissionError:
                                print('Unable to copy parameter file. May already be saved.')
                    except FileNotFoundError:
                        print(f'Unable to find {app} file in folder.')
                        temp = 0
                except UnboundLocalError:
                    temp = 0
                if isinstance(temp, (np.ndarray, list)) == True:
                    try:
                        temp = temp[-1]
                        temp = round(temp,2)
                    except IndexError:
                        temp = temp
                        temp = round(float(temp),2)
                else:
                    temp = temp
                    temp = round(float(temp),2)
                if save == True:
                    try:
                        shutil.copy(dire,f'{newdir}')
                    except PermissionError:
                        print('Unable to copy file. May already be saved.')
                if check_four.exists():
                    dire = testdir_four
                return dire, temp, sumSpec
            elif check_three.exists():
                dire = testdir_three
                name = f'ENGINX00{run}'
                dSpace = f'{name}-dSpacing'
                sumSpec = f'{name}-sumSpec'
                try:
                    Load(Filename=dire, OutputWorkspace=name, SpectrumMin=sMin_One, SpectrumMax=sMax_One)
                except ValueError:
                    Load(Filename=dire, OutputWorkspace=name, SpectrumMin=sMin_Two, SpectrumMax=sMax_Two)
                ConvertUnits(InputWorkspace=name, OutputWorkspace=dSpace, Target='dSpacing', AlignBins=True)
                SumSpectra(InputWorkspace=dSpace, OutputWorkspace=sumSpec)
                try:
                    temp = mtd[sumSpec].getRun().getLogData(app).filtered_value
                except RuntimeError:
                    print(f'Unable to find {app} in file.')
                    try:
                        temp_fp = f'//isis/inst$/NDXENGINX/Instrument/data/cycle_{year}_{cycle}/{name_one}_{app}.txt'
                        temp = np.loadtxt(fname=temp_fp,dtype='float',usecols=(1))
                        if save == True:
                            try:
                                shutil.copy(temp_fp,f'{newdir}')
                            except PermissionError:
                                print('Unable to copy parameter file. May already be saved.')
                    except FileNotFoundError:
                        print(f'Unable to find {app} file in folder.')
                        temp = 0
                except UnboundLocalError:
                    temp = 0
                if isinstance(temp, (np.ndarray, list)) == True:
                    try:
                        temp = temp[-1]
                        temp = round(temp,2)
                    except IndexError:
                        temp = temp
                        temp = round(float(temp),2)
                else:
                    temp = temp
                    temp = round(float(temp),2)
                if save == True:
                    try:
                        shutil.copy(dire,f'{newdir}')
                    except PermissionError:
                        print('Unable to copy file. May already be saved.')
                return dire, temp, sumSpec
            elif check_four.exists():
                dire = testdir_four
                name = f'ENGINX00{run}'
                dSpace = f'{name}-dSpacing'
                sumSpec = f'{name}-sumSpec'
                try:
                    Load(Filename=dire, OutputWorkspace=name, SpectrumMin=sMin_One, SpectrumMax=sMax_One)
                except ValueError:
                    Load(Filename=dire, OutputWorkspace=name, SpectrumMin=sMin_Two, SpectrumMax=sMax_Two)
                ConvertUnits(InputWorkspace=name, OutputWorkspace=dSpace, Target='dSpacing', AlignBins=True)
                SumSpectra(InputWorkspace=dSpace, OutputWorkspace=sumSpec)
                try:
                    temp = mtd[sumSpec].getRun().getLogData(app).filtered_value
                except RuntimeError:
                    print(f'Unable to find {app} in file.')
                    try:
                        temp_fp = f'//isis/inst$/NDXENGINX/Instrument/data/cycle_{year}_{cycle}/{name_two}_{app}.txt'
                        temp = np.loadtxt(fname=temp_fp,dtype='float',usecols=(1))
                        if save == True:
                            try:
                                shutil.copy(temp_fp,f'{newdir}')
                            except PermissionError:
                                print('Unable to copy parameter file. May already be saved.')
                    except FileNotFoundError:
                        print(f'Unable to find {app} file in folder.')
                        temp = 0
                except UnboundLocalError:
                    temp = 0
                if isinstance(temp, (np.ndarray, list)) == True:
                    try:
                        temp = temp[-1]
                        temp = round(temp,2)
                    except IndexError:
                        temp = temp
                        temp = round(float(temp),2)
                else:
                    temp = temp
                    temp = round(float(temp),2)
                if save == True:
                    try:
                        shutil.copy(dire,f'{newdir}')
                    except PermissionError:
                        print('Unable to copy file. May already be saved.')
                return dire, temp, sumSpec
            else:
                pass

In [2]:
def localFileSearch(path,run,exp,bank,save=False):
    if run < 269853:
        app_year = False
    else:
        app_year = True
    if run < 99406:
        name_year = False
    else:
        name_year = True
    exps = ['Cryo','Furnace','Pressure','X Scan','Y Scan','Z Scan']
    currdir = os.getcwd() #This finds the folder that the script is being ran in
    savedir = os.path.join(currdir,'Saves') #This creates a new folder to save the data in
    newdir = os.path.join(savedir,'Runs')
    if not os.path.exists(newdir):
        os.makedirs(newdir)
    filedir = os.path.join(newdir,f'{run}')
    
    if len(str(run)) < 5:
        zero_diff = 5 - len(str(run))
        for i in range(zero_diff):
            run = f'0{run}' #This is for significantly early runs, such as 00001 and 00035 from 2002
            
    yr = dt.now().strftime('%y')
    cy = int(yr) + 1

    if bank == '1':
        sMin_One = 0
        sMax_One = 1200 #These are the spectrums to be provided to the load function. After a certain point,
        #the highest and lowest spectrum available to read are 1 and 2399 respectively, so this code factors
        #that in
        sMin_Two = 1
        sMax_Two = 1199
    elif bank == '2':
        sMin_One = 1201
        sMax_One = 2400
        sMin_Two = 1200
        sMax_Two = 2399
    elif bank == 'Both':
        sMin_One = 0
        sMax_One = 2400
        sMin_Two = 1
        sMax_Two = 2399
    else:
        sMin_One = 0
        sMax_One = 2400
        sMin_Two = 1
        sMax_Two = 2399
    if app_year == False:
        #These are the names of all the parameters saved as part of the raw file or folder prior
        #to the apparent overhaul in 2017
        apps = [['cryo_1','cryo_temp1'],'furnace','position','x','y','z']
        if exp == exps[0]:
            if i < 10:
                app = apps[0][0]
            else:
                app = apps[0][1]
        elif exp == exps[1]:
            app = apps[1]
        elif exp == exps[2]:
            app = apps[2]
        elif exp == exps[3]:
            app = apps[3]
        elif exp == exps[4]:
            app = apps[4]
        elif exp == exps[5]:
            app = apps[5]
        elif exp == None:
            pass
        else:
            raise NameError(f'This is not included in the list of experiments. Please enter one from this list: {exps}')
    else:
        #These are the current names for the parameters found within the nexus files
        apps = ['Temp_1','Temp_3','position','X_position','Y_position','Z_position']
        if exp == exps[0]:
            app = apps[0]
        elif exp == exps[1]:
            app = apps[1]
        elif exp == exps[2]:
            app = apps[2]
        elif exp == exps[3]:
            app = apps[3]
        elif exp == exps[4]:
            app = apps[4]
        elif exp == exps[5]:
            app = apps[5]
        elif exp == None:
            pass
        else:
            raise NameError(f'This is not included in the list of experiments. Please enter one from this list: {exps}')
    if name_year == False:
        name_one = f'ENG{run}'
        name_two = f'ENG{run}' #This factors in a change in naming convention
    else:
        name_one = f'ENG00{run}'
        name_two = f'ENGINX00{run}'
    ext_one = '.raw'
    ext_two = '.nxs'
    #These four combinations factor in the naming convention of runs mentioned above and the filetype,
    #either nexus or raw
    testdir_one = f'{path}/{name_one}{ext_one}'
    testdir_two = f'{path}/{name_two}{ext_one}'
    testdir_three = f'{path}/{name_one}{ext_two}'
    testdir_four = f'{path}/{name_two}{ext_two}'
    check_one = pathlib.Path(testdir_one)
    check_two = pathlib.Path(testdir_two)
    check_three = pathlib.Path(testdir_three)
    check_four = pathlib.Path(testdir_four)
    if check_one.exists():
        dire = testdir_one
        name = f'ENGINX00{run}'
        dSpace = f'{name}-dSpacing'
        sumSpec = f'{name}-sumSpec'
        try:
            Load(Filename=dire, OutputWorkspace=name, SpectrumMin=sMin_One, SpectrumMax=sMax_One)
        except ValueError:
            Load(Filename=dire, OutputWorkspace=name, SpectrumMin=sMin_Two, SpectrumMax=sMax_Two) #For the
            #different spectrums, mentioned earlier
        ConvertUnits(InputWorkspace=name, OutputWorkspace=dSpace, Target='dSpacing', AlignBins=True)
        SumSpectra(InputWorkspace=dSpace, OutputWorkspace=sumSpec)
        try:
            #It searches for the parameter in the file
            temp = mtd[sumSpec].getRun().getLogData(app).filtered_value
        except RuntimeError:
            print(f'Unable to find {app} in file.')
            try:
                #If this doesn't work, it searches for the parameter in the folder
                temp_fp = f'{path}/{name_one}_{app}.txt'
                temp = np.loadtxt(fname=temp_fp,dtype='float',usecols=(1))
                if save == True:
                    try:
                        shutil.copy(temp_fp,f'{newdir}')
                    except PermissionError:
                        print('Unable to copy parameter file. May already be saved.')
            except FileNotFoundError:
                #If this still doesn't work, it notifies the user and carries on
                print(f'Unable to find {app} file in folder.')
                temp = 0
        except UnboundLocalError:
            temp = 0
        if isinstance(temp, (np.ndarray, list)) == True:
            try:
                #This is due to the nature of the txt files some parameters are read from holding more
                #information than just the parameter the user would like to retrieve
                temp = temp[-1]
                temp = round(temp,2)
            except IndexError:
                temp = temp
                temp = round(float(temp),2)
        else:
            temp = temp
            temp = round(float(temp),2)
        if save == True:
            try:
                shutil.copy(dire,f'{newdir}')
            except PermissionError:
                print('Unable to copy file. May already be saved.')
        return dire, temp, sumSpec
    elif check_two.exists():
        #It then repeats this process for every potential name till it finds a fit
        dire = testdir_two
        name = f'ENGINX00{run}'
        dSpace = f'{name}-dSpacing'
        sumSpec = f'{name}-sumSpec'
        try:
            Load(Filename=dire, OutputWorkspace=name, SpectrumMin=sMin_One, SpectrumMax=sMax_One)
        except ValueError:
            Load(Filename=dire, OutputWorkspace=name, SpectrumMin=sMin_Two, SpectrumMax=sMax_Two)
        ConvertUnits(InputWorkspace=name, OutputWorkspace=dSpace, Target='dSpacing', AlignBins=True)
        SumSpectra(InputWorkspace=dSpace, OutputWorkspace=sumSpec)
        try:
            temp = mtd[sumSpec].getRun().getLogData(app).filtered_value
        except RuntimeError:
            print(f'Unable to find {app} in file.')
            try:
                temp_fp = f'{path}/{name_two}_{app}.txt'
                temp = np.loadtxt(fname=temp_fp,dtype='float',usecols=(1))
                if save == True:
                    try:
                        shutil.copy(temp_fp,f'{newdir}')
                    except PermissionError:
                        print('Unable to copy parameter file. May already be saved.')
            except FileNotFoundError:
                print(f'Unable to find {app} file in folder.')
                temp = 0
        except UnboundLocalError:
            temp = 0
        if isinstance(temp, (np.ndarray, list)) == True:
            try:
                temp = temp[-1]
                temp = round(temp,2)
            except IndexError:
                temp = temp
                temp = round(float(temp),2)
        else:
            temp = temp
            temp = round(float(temp),2)
        if save == True:
            try:
                shutil.copy(dire,f'{newdir}')
            except PermissionError:
                print('Unable to copy file. May already be saved.')
        if check_four.exists():
            dire = testdir_four
        return dire, temp, sumSpec
    elif check_three.exists():
        dire = testdir_three
        name = f'ENGINX00{run}'
        dSpace = f'{name}-dSpacing'
        sumSpec = f'{name}-sumSpec'
        try:
            Load(Filename=dire, OutputWorkspace=name, SpectrumMin=sMin_One, SpectrumMax=sMax_One)
        except ValueError:
            Load(Filename=dire, OutputWorkspace=name, SpectrumMin=sMin_Two, SpectrumMax=sMax_Two)
        ConvertUnits(InputWorkspace=name, OutputWorkspace=dSpace, Target='dSpacing', AlignBins=True)
        SumSpectra(InputWorkspace=dSpace, OutputWorkspace=sumSpec)
        try:
            temp = mtd[sumSpec].getRun().getLogData(app).filtered_value
        except RuntimeError:
            print(f'Unable to find {app} in file.')
            try:
                temp_fp = f'{path}/{name_one}_{app}.txt'
                temp = np.loadtxt(fname=temp_fp,dtype='float',usecols=(1))
                if save == True:
                    try:
                        shutil.copy(temp_fp,f'{newdir}')
                    except PermissionError:
                        print('Unable to copy parameter file. May already be saved.')
            except FileNotFoundError:
                print(f'Unable to find {app} file in folder.')
                temp = 0
        except UnboundLocalError:
            temp = 0
        if isinstance(temp, (np.ndarray, list)) == True:
            try:
                temp = temp[-1]
                temp = round(temp,2)
            except IndexError:
                temp = temp
                temp = round(float(temp),2)
        else:
            temp = temp
            temp = round(float(temp),2)
        if save == True:
            try:
                shutil.copy(dire,f'{newdir}')
            except PermissionError:
                print('Unable to copy file. May already be saved.')
        return dire, temp, sumSpec
    elif check_four.exists():
        dire = testdir_four
        name = f'ENGINX00{run}'
        dSpace = f'{name}-dSpacing'
        sumSpec = f'{name}-sumSpec'
        try:
            Load(Filename=dire, OutputWorkspace=name, SpectrumMin=sMin_One, SpectrumMax=sMax_One)
        except ValueError:
            Load(Filename=dire, OutputWorkspace=name, SpectrumMin=sMin_Two, SpectrumMax=sMax_Two)
        ConvertUnits(InputWorkspace=name, OutputWorkspace=dSpace, Target='dSpacing', AlignBins=True)
        SumSpectra(InputWorkspace=dSpace, OutputWorkspace=sumSpec)
        try:
            temp = mtd[sumSpec].getRun().getLogData(app).filtered_value
        except RuntimeError:
            print(f'Unable to find {app} in file.')
            try:
                temp_fp = f'{path}/{name_two}_{app}.txt'
                temp = np.loadtxt(fname=temp_fp,dtype='float',usecols=(1))
                if save == True:
                    try:
                        shutil.copy(temp_fp,f'{newdir}')
                    except PermissionError:
                        print('Unable to copy parameter file. May already be saved.')
            except FileNotFoundError:
                print(f'Unable to find {app} file in folder.')
                temp = 0
        except UnboundLocalError:
            temp = 0
        if isinstance(temp, (np.ndarray, list)) == True:
            try:
                temp = temp[-1]
                temp = round(temp,2)
            except IndexError:
                temp = temp
                temp = round(float(temp),2)
        else:
            temp = temp
            temp = round(float(temp),2)
        if save == True:
            try:
                shutil.copy(dire,f'{newdir}')
            except PermissionError:
                print('Unable to copy file. May already be saved.')
        return dire, temp, sumSpec
    else:
        pass

In [ ]:
dire,temp,sumSpec = fileSearch(346540,exp='Cryo',bank=None,save=False,local=True,path=r'C:\Users\ynn46697\Documents\ISIS\For_GitHub\Misc\Saves\Runs')
print(dire,temp,sumSpec)